# Plan and Execute for Deep Web Search 

- This code is designed to plan and execute a search on the deep web using a specified search engine.


In [1]:

import sys
import os
from openai import AzureOpenAI
import requests
import json
from urllib.parse import urljoin
from langchain_core.prompts import PromptTemplate
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from langchain_core.prompts import load_prompt
import sys
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from utils.search_utils import web_search, url_search, extract_contexts_async
from IPython.display import Markdown, display
from datetime import datetime
import pytz
import time
import os
from dotenv import load_dotenv
load_dotenv(override=True) 
import logging

logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


# Get credentials from environment variables
BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")
BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 10))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "gpt-4o-mini")

# Web search mode: "google" or "bing"
web_search_mode = "bing"


client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-08-01-preview"
)

current_dir = os.getcwd()

def rewrite_query_for_search(query, client: AzureOpenAI):
    
  prompt_path = os.path.join(current_dir, "prompts", "rewrite_prompt.yaml")
  QUERY_REWRITE_PROMPT = load_prompt(prompt_path, encoding="utf-8")
  response = client.chat.completions.create(
      model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
      messages=[
          {"role": "system", "content": QUERY_REWRITE_PROMPT.format(
            user_query=query)},
          {"role": "user", "content": query}
      ],
      temperature=0.8,
      max_tokens=300,
      response_format= {"type": "json_object"},
  )
  
  return json.loads(response.choices[0].message.content.strip())
    
  
def plan_query_for_search(query, client: AzureOpenAI):

  current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")
  
  prompt_path = os.path.join(current_dir, "prompts", "planner_prompt.yaml")
  PLANNER_PROMPT = load_prompt(prompt_path, encoding="utf-8")
  
  response = client.chat.completions.create(
      model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
      messages=[
          {"role": "system", "content": PLANNER_PROMPT.format(current_date=current_date, query=query)},
          {"role": "user", "content": query}
      ],
      temperature=0.8,
      max_tokens=300,
      response_format= {"type": "json_object"},
  )
  
  return json.loads(response.choices[0].message.content.strip())


In [2]:


   

# 웹 검색 결과를 활용해 LLM 답변을 생성하는 비동기 함수
async def process_deep_search(max_result_count=3, input_query=None, web_search_mode=None, product_name=None):
    
    start_time = time.time()
    print(f"Original Input: {input_query}")

    # 검색 모드가 지정되지 않으면 환경변수
    if web_search_mode is None:
        web_search_mode = os.getenv("WEB_SEARCH_MODE", "google").lower()
    print(f"############## Web Search Mode: {web_search_mode}")
    
    # query rewrite (검색용/LLM용)
    query_rewrite = rewrite_query_for_search(input_query, client)
    print(f"Web Search Query: {query_rewrite['search_query']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    # plan deep query
    search_plan_json = plan_query_for_search(query_rewrite['search_query'], client)
    print(f"Planned Search Queries: {search_plan_json}")
    search_queries = search_plan_json.get("search_queries", [])
    all_contexts = []
    
    for i, query in enumerate(search_queries):
        print(f"Processing query {i+1}: {query}")
        search_results = url_search(query, max_result_count, web_search_mode=web_search_mode, product_name=product_name)
        
        print(f"Search Results: {len(search_results)} results found for query '{query}'")
        
        print("Analyze search results...")        
        if search_results:
            url_snippet_tuples = [(r["link"], r["snippet"]) for r in search_results]
            contexts = await extract_contexts_async(url_snippet_tuples=url_snippet_tuples)
            
            formatted_contexts = [
                f"[search keyword: {query}]\n{context}"
                for context in contexts
            ]
            
            all_contexts.extend(formatted_contexts)
        
    
    
    current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")
        
    # Get the current working directory for this notebook
    prompt_path = os.path.join(current_dir, "prompts", "generate_prompt.yaml")
    GENERATE_PROMPT = load_prompt(prompt_path, encoding="utf-8")
    
    print("Generate response...")            
    
    answer_messages = [
        {"role": "system", "content": GENERATE_PROMPT.format(
            product_name=product_name,
            date=current_date,
            contexts=all_contexts,
            user_query=query_rewrite['llm_query'],
        )},
        {"role": "user", "content": query_rewrite['llm_query']}
    ]
    
    response = client.chat.completions.create(
        model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
        messages=answer_messages,
        top_p=0.9,
        max_tokens=1500
    )
    display(Markdown(response.choices[0].message.content))
    
    end_time = time.time()
    print(f"elapsed time: {end_time - start_time:.2f} seconds\n\n")


In [4]:
RESULTS_COUNT = 5

inputs = [
    "티뷰론과 아이오닉5를 비교해줘"
]

web_search_mode = "google"

for input in inputs:
    print(f"Bing Grounding 검색 사용: {input}")
    await process_deep_search(max_result_count=RESULTS_COUNT, input_query=input, web_search_mode=web_search_mode, product_name="현대자동차")  # product_name은 필요에 따라 변경 가능

Bing Grounding 검색 사용: 티뷰론과 아이오닉5를 비교해줘
Original Input: 티뷰론과 아이오닉5를 비교해줘
############## Web Search Mode: google


Web Search Query: 티뷰론 아이오닉5 비교
LLM Query: 티뷰론과 아이오닉5를 다양한 측면에서 비교해 주세요. 성능, 디자인, 가격, 연비 등 각 차종의 특징과 장단점에 대해 자세히 설명해 주시면 감사하겠습니다.
Planned Search Queries: {'search_queries': ['티뷰론 아이오닉5 비교', '티뷰론 vs 아이오닉5', '아이오닉5 성능 특징']}
Processing query 1: 티뷰론 아이오닉5 비교


2025-06-24 05:50:08,457 - utils.search_utils - ERROR - Request failed: Client error '403 Forbidden' for url 'https://namu.wiki/w/%ED%98%84%EB%8C%80%20%EC%95%84%EC%9D%B4%EC%98%A4%EB%8B%89%205'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403


Search Results: 5 results found for query '티뷰론 아이오닉5 비교'
Analyze search results...
Processing query 2: 티뷰론 vs 아이오닉5
Search Results: 5 results found for query '티뷰론 vs 아이오닉5'
Analyze search results...
Processing query 3: 아이오닉5 성능 특징
Search Results: 5 results found for query '아이오닉5 성능 특징'
Analyze search results...


2025-06-24 05:50:19,028 - utils.search_utils - ERROR - Request failed: Client error '403 Forbidden' for url 'https://namu.wiki/w/%ED%98%84%EB%8C%80%20%EC%95%84%EC%9D%B4%EC%98%A4%EB%8B%89%205'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403


Generate response...


티뷰론과 아이오닉 5는 각각 현대자동차의 독특한 모델로, 성격과 특징이 많이 다릅니다. 아래는 두 모델을 성능, 디자인, 가격, 연비 등 다양한 측면에서 비교한 내용입니다.

### 1. 성능
- **티뷰론**
  - **엔진**: 티뷰론은 2.0 리터 베타 엔진을 장착하여 최대 출력 약 150마력을 발휘합니다.
  - **주행 성능**: 날렵한 디자인과 경량화된 차체 덕분에 스포티한 주행 감각을 제공합니다. 경량 스포츠카의 특성을 잘 반영하여 핸들링이 우수합니다.
  
- **아이오닉 5**
  - **전기 모터**: 아이오닉 5는 다양한 배터리 옵션을 제공하며, 롱레인지 모델의 경우 후륜구동 기준으로 약 225마력의 출력을 가지고 있습니다.
  - **주행 성능**: 0-100km/h 가속시간이 약 5.2초로 빠르며, 뛰어난 주행 안정성과 승차감을 제공합니다. 특히 800V 초고속 충전 시스템을 통해 충전 속도가 매우 빠릅니다.

### 2. 디자인
- **티뷰론**
  - **디자인 특징**: 쿠페형 실루엣으로 스포티하고 역동적인 디자인이 특징입니다. 날렵한 외관이 젊은 세대의 감성을 자극하며, 당시로서는 혁신적인 디자인으로 주목받았습니다.
  
- **아이오닉 5**
  - **디자인 특징**: 레트로와 미래지향적인 디자인을 혼합한 스타일로, 독창적인 헤드램프와 넓은 실내공간이 인상적입니다. E-GMP 플랫폼을 기반으로 한 디자인으로 실용성을 강조했습니다.

### 3. 가격
- **티뷰론**
  - **가격대**: 중고 시장에서 주로 거래되며, 상태에 따라 가격이 다양합니다. 초기 출시 가격은 대략 1,500만 원대였으나, 현재는 복원 프로젝트 등을 통해 가격이 상승할 수 있습니다.
  
- **아이오닉 5**
  - **가격대**: 현재 신차 가격은 약 4,086만 원에서 5,755만 원까지 다양합니다. 최신 전기차 기술이 적용된 만큼 상대적으로 높은 가격대입니다.

### 4. 연비
- **티뷰론**
  - **연비**: 대략 10-12km/l의 연비를 제공합니다. 스포츠카의 특성상 연비는 다소 아쉬울 수 있습니다.
  
- **아이오닉 5**
  - **연비**: 복합 기준으로 약 4.5km/kWh를 기록하며, 전기차 특성상 효율성이 매우 높습니다. 72.6kWh 배터리를 장착한 롱레인지 모델 기준으로 최대 485km의 주행이 가능합니다.

### 5. 장단점 요약
| 차종      | 장점                                           | 단점                           |
|-----------|----------------------------------------------|-------------------------------|
| 티뷰론   | - 스포티한 디자인<br>- 우수한 핸들링 성능     | - 연비가 낮음<br>- 중고 시장에서의 가용성 문제 |
| 아이오닉 5 | - 뛰어난 주행 효율성과 충전 속도<br>- 혁신적인 디자인 | - 상대적으로 높은 가격대     |

결론적으로, 티뷰론은 과거의 감성을 자극하는 스포츠카로, 주로 레트로 디자인과 핸들링 성능에 강점을 두고 있습니다. 반면 아이오닉 5는 현대 전기차의 대표주자로, 효율성과 실용성을 중시하며 최신 기술이 적용된 모델입니다. 각 차종의 특성을 고려하여 구매 결정을 내리는 것이 좋습니다.

elapsed time: 38.32 seconds


